In [ ]:
from understatapi import UnderstatClient
import pandas as pd
import numpy as np
from math import hypot
from time import sleep
from tqdm.auto import tqdm
import json
from pprint import pprint

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

In [ ]:
LEAGUE = "EPL"    # "EPL", "La_Liga", "Bundesliga", "Serie_A", "Ligue_1", "RFPL"
SEASON = ["2025"]

In [ ]:
def get_league_match_ids(league: str, seasons: list[str]):
    """Retorna lista de match_ids por liga y temporadas."""
    all_match_ids = []
    all_matches = []

    with UnderstatClient() as us:
        for season in seasons:
            matches = us.league(league=league).get_match_data(season=season)
            all_match_ids.extend([m["id"] for m in matches])
            all_matches.extend(matches)

    return all_match_ids, all_matches

match_ids, raw_matches = get_league_match_ids(LEAGUE, SEASON)
print(f"Total de matches encontrados: {len(match_ids)}")
print("Ejemplos de match_ids:", match_ids[:10])

print("\nEjemplo de cómo viene un match:")
pprint({k: raw_matches[0][k] for k in list(raw_matches[0].keys())[:10]})

In [ ]:
# METODOS de obtención y formateo de datos Optimizados
def fetch_match_shots(match_id: str, client: UnderstatClient) -> list[dict]:
    out = []
    md = client.match(match=match_id).get_shot_data()
    for side in ("h", "a"):
        for s in md.get(side, []):
            s = dict(s)  # copy
            s["h_a"] = side
            s["match_id"] = match_id
            out.append(s)
    return out

def shots_to_df(shots: list[dict]) -> pd.DataFrame:
    rows = []
    for s in shots:
        rows.append({
            "match_id": s.get("match_id"),
            "minute": int(s.get("minute")),
            "team": s.get("h_team") if s.get("h_a") == "h" else s.get("a_team"),
            "h_a": s.get("h_a"),
            "player": s.get("player"),
            "result": s.get("result"),             
            "X": float(s.get("X")),                
            "Y": float(s.get("Y")),                
            "is_goal": 1 if s.get("result") == "Goal" else 0,
            "situation": s.get("situation")
        })
    df = pd.DataFrame(rows)
    return df.dropna(subset=["X", "Y"]).reset_index(drop=True)

In [ ]:
all_shots = []
with UnderstatClient() as us:
    for mid in tqdm(match_ids, desc=f"Descargando tiros {LEAGUE} {SEASON}"):
        try:
            all_shots.extend(fetch_match_shots(mid, us))
        except Exception:
            pass
        sleep(0.15)

len(all_shots)

In [ ]:
# --- Dataframe preview ---
df_raw = shots_to_df(all_shots)

print("Rows:", len(df_raw))

df_raw = df_raw[~df_raw["situation"].isin(["Penalty","DirectFreekick"])]

print("Rows filtrando Penalties y tiros libres:", len(df_raw))

df_raw.sample(8).sort_values(["match_id","minute"]).head(8)


In [ ]:
# --- Estandarizar coordenadas ---
def standardize_shot_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    mask = out["X"] < 0.5
    out.loc[mask, "X"] = 1 - out.loc[mask, "X"]
    return out

df_std = standardize_shot_coordinates(df_raw)


In [ ]:
PITCH_LENGTH_M = 100.0
PITCH_WIDTH_M = 65.0

def add_longitudinal_distance_m(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Con X estandarizado hacia x=1: la línea de gol rival está en X=1
    out["dist_long_m"] = (1.0 - out["X"]) * PITCH_LENGTH_M
    return out

df_long = add_longitudinal_distance_m(df_std)

In [ ]:
BIN_WIDTH = 1.0
ALPHA = 1.0
LOW_N = 10
D_MIN, D_MAX = 2.0, 33.0

bins = np.arange(0.0, np.ceil(df_long["dist_long_m"].max()) + BIN_WIDTH, BIN_WIDTH)
df_long["dist_bin"] = pd.cut(df_long["dist_long_m"], bins=bins, right=False)

bin_stats_long = (df_long
    .groupby("dist_bin", observed=True)
    .agg(
        shots=("is_goal","size"),
        goals=("is_goal","sum"),
        d_min=("dist_long_m","min"),
        d_max=("dist_long_m","max")
    )
    .dropna(subset=["d_min","d_max"])
    .reset_index(drop=True)
)

bin_stats_long["d_mid"]  = 0.5*(bin_stats_long["d_min"]+bin_stats_long["d_max"])
bin_stats_long["p_goal"] = (bin_stats_long["goals"] + ALPHA) / (bin_stats_long["shots"] + 2*ALPHA)

In [ ]:
# Muestra dispersión de rango completo con colores para colas "ruidosas" y bins de muestra bajos
LOW_N = 10

tail_mask = bin_stats_long["d_mid"] > D_MAX
lown_mask = bin_stats_long["shots"] < LOW_N

plt.figure(figsize=(8,5))
plt.scatter(bin_stats_long["d_mid"], bin_stats_long["p_goal"], s=24, label="Todos los bins")
plt.scatter(bin_stats_long.loc[tail_mask, "d_mid"],
            bin_stats_long.loc[tail_mask, "p_goal"], s=36, label=f"> {D_MAX} m (cola)")
plt.scatter(bin_stats_long.loc[lown_mask, "d_mid"],
            bin_stats_long.loc[lown_mask, "p_goal"], s=36, label=f"tiros < {LOW_N}")
plt.axvline(D_MAX, ls="--", lw=1.5)
plt.xlabel("Distancia hasta la portería (m)")
plt.ylabel("Probabilidad de Gol calculada")
plt.title("bins ruidosos al graficar >30m")
plt.legend()
plt.show()


In [ ]:
# Cuantificamos porqué los bins son ruidosos
share_tail = (df_long["dist_long_m"] > D_MAX).mean()
shots_tail = int((df_long["dist_long_m"] > D_MAX).sum())
print(f"Tiros más allá de {D_MAX} m: {shots_tail} ({share_tail:.1%} de todos los tiros)")

# Mostrar los peores: bins con p alto pero tamaños de muestra pequeños o distancias muy largas
display_cols = ["d_mid", "shots", "goals", "p_goal"]
tail_preview = (bin_stats_long[tail_mask | lown_mask]
                .sort_values(["d_mid", "shots"], ascending=[True, True])
                [display_cols].head(12))
tail_preview


In [ ]:
fit_bins = bin_stats_long[
    (bin_stats_long["shots"] >= LOW_N) &
    (bin_stats_long["d_mid"] >= D_MIN) &
    (bin_stats_long["d_mid"] <= D_MAX)
].copy()

plt.figure(figsize=(8,5))
plt.scatter(bin_stats_long["d_mid"], bin_stats_long["p_goal"], s=18, alpha=0.35, label="Todos los bins")
plt.scatter(fit_bins["d_mid"], fit_bins["p_goal"], s=36, label="Conservamos para la regresión")
plt.axvspan(D_MIN, D_MAX, color="grey", alpha=0.1, label="Rango de regresión")
plt.xlabel("Distancia hasta la portería (m)")
plt.ylabel("Probabilidad de gol por tiro")
plt.title("Rango de regresión limpio: 2–30 m, bins con ≥10 tiros")
plt.legend()
plt.show()


In [ ]:
X_log = np.log(fit_bins["d_mid"].values).reshape(-1,1)
y_log = np.log(fit_bins["p_goal"].values)
w = fit_bins["shots"].values

lin = LinearRegression()
lin.fit(X_log, y_log, sample_weight=w)
b_hat = lin.coef_[0]
a_hat = float(np.exp(lin.intercept_))

In [ ]:
def powercurve_xg(d_array, a: float, b: float):
    """power curve p(d) = a * d^b vectorizada."""
    d = np.maximum(np.asarray(d_array, dtype=float), 0.5)
    return a * (d ** b)

d_all = np.linspace(bin_stats_long["d_mid"].min(), bin_stats_long["d_mid"].max(), 400)
d_fit = np.linspace(D_MIN, D_MAX, 300)

plt.figure(figsize=(8,5))
plt.scatter(bin_stats_long["d_mid"], bin_stats_long["p_goal"], s=22, alpha=0.28, label="All bins")
plt.scatter(fit_bins["d_mid"], fit_bins["p_goal"], s=40, alpha=0.95, label="Kept for fit")
plt.axvspan(D_MIN, D_MAX, color="grey", alpha=0.10, label="Fit range")

p_fit_wls = powercurve_xg(d_fit, a_hat, b_hat)
plt.plot(d_fit, p_fit_wls, lw=2, label=f"WLS (bins) curve (a={a_hat:.4f}, b={b_hat:.4f})")

plt.xlabel("Distancia hasta la portería (m)")
plt.ylabel("Probabilidad de gol por tiro")
plt.title(f"Curva de potencia — {LEAGUE} {SEASON}")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Plain R² (bins)
p_pred_bin = np.clip(powercurve_xg(fit_bins["d_mid"].values, a_hat, b_hat), 1e-9, 1-1e-9)
r2_bin = r2_score(fit_bins["p_goal"].values, p_pred_bin)
print(f"a={a_hat:.6f}, b={b_hat:.6f}, R² (bin, longitudinal) = {r2_bin:.3f}")

# Mejora: Distancia Euclidiana (X + Y)

Usar la **distancia total** al centro de la portería (combinando X e Y con Pitágoras) en lugar de solo la distancia longitudinal puede mejorar significativamente el modelo.

La fórmula es: `dist_total = sqrt(dx² + dy²)`

Donde:
- `dx` = distancia longitudinal a la portería (en metros)
- `dy` = distancia lateral al centro de la portería (en metros)

In [ ]:
GOAL_CENTER_Y = 0.5

def add_euclidean_distance_m(df: pd.DataFrame) -> pd.DataFrame:
    """Calcula la distancia euclidiana total al centro de la portería."""
    out = df.copy()
    dx = out["dist_long_m"]
    dy = np.abs(out["Y"] - GOAL_CENTER_Y) * PITCH_WIDTH_M
    out["dist_lateral_m"] = dy
    out["dist_euclid_m"] = np.sqrt(dx**2 + dy**2)
    return out

df_euclid = add_euclidean_distance_m(df_long)

print("Comparación de distancias (muestra aleatoria):")
df_euclid[["X", "Y", "dist_long_m", "dist_lateral_m", "dist_euclid_m"]].sample(5).round(2)

In [ ]:
# Binning con distancia euclidiana
bins_euclid = np.arange(0.0, np.ceil(df_euclid["dist_euclid_m"].max()) + BIN_WIDTH, BIN_WIDTH)
df_euclid["dist_bin_euclid"] = pd.cut(df_euclid["dist_euclid_m"], bins=bins_euclid, right=False)

bin_stats_euclid = (df_euclid
    .groupby("dist_bin_euclid", observed=True)
    .agg(
        shots=("is_goal", "size"),
        goals=("is_goal", "sum"),
        d_min=("dist_euclid_m", "min"),
        d_max=("dist_euclid_m", "max")
    )
    .dropna(subset=["d_min", "d_max"])
    .reset_index(drop=True)
)

bin_stats_euclid["d_mid"] = 0.5 * (bin_stats_euclid["d_min"] + bin_stats_euclid["d_max"])
bin_stats_euclid["p_goal"] = (bin_stats_euclid["goals"] + ALPHA) / (bin_stats_euclid["shots"] + 2 * ALPHA)

# Filtrar bins para regresión
fit_bins_euclid = bin_stats_euclid[
    (bin_stats_euclid["shots"] >= LOW_N) &
    (bin_stats_euclid["d_mid"] >= D_MIN) &
    (bin_stats_euclid["d_mid"] <= D_MAX)
].copy()

print(f"Bins para regresión euclidiana: {len(fit_bins_euclid)}")

In [ ]:
# Ajuste de curva de potencia con distancia euclidiana
X_log_euclid = np.log(fit_bins_euclid["d_mid"].values).reshape(-1, 1)
y_log_euclid = np.log(fit_bins_euclid["p_goal"].values)
w_euclid = fit_bins_euclid["shots"].values

lin_euclid = LinearRegression()
lin_euclid.fit(X_log_euclid, y_log_euclid, sample_weight=w_euclid)
b_hat_euclid = lin_euclid.coef_[0]
a_hat_euclid = float(np.exp(lin_euclid.intercept_))

print(f"Parámetros curva euclidiana: a={a_hat_euclid:.6f}, b={b_hat_euclid:.6f}")

In [ ]:
# Comparación de R² entre ambos modelos

# R² modelo longitudinal (ya calculado arriba)
p_pred_long = np.clip(powercurve_xg(fit_bins["d_mid"].values, a_hat, b_hat), 1e-9, 1-1e-9)
r2_long = r2_score(fit_bins["p_goal"].values, p_pred_long)

# R² modelo euclidiano
p_pred_euclid = np.clip(powercurve_xg(fit_bins_euclid["d_mid"].values, a_hat_euclid, b_hat_euclid), 1e-9, 1-1e-9)
r2_euclid = r2_score(fit_bins_euclid["p_goal"].values, p_pred_euclid)

print("=" * 50)
print("COMPARACIÓN DE MODELOS")
print("=" * 50)
print(f"\nModelo Longitudinal (solo X):")
print(f"  xG = {a_hat:.4f} × dist^{b_hat:.4f}")
print(f"  R² = {r2_long:.3f}")
print(f"\nModelo Euclidiano (X + Y):")
print(f"  xG = {a_hat_euclid:.4f} × dist^{b_hat_euclid:.4f}")
print(f"  R² = {r2_euclid:.3f}")
print(f"\nMejora en R²: +{(r2_euclid - r2_long):.3f} ({((r2_euclid - r2_long) / r2_long * 100):.1f}%)")

In [ ]:
# Visualización comparativa de ambos modelos
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

d_range = np.linspace(D_MIN, D_MAX, 300)

# Panel izquierdo: Modelo Longitudinal
ax1 = axes[0]
ax1.scatter(fit_bins["d_mid"], fit_bins["p_goal"], s=40, alpha=0.7, label="Datos observados")
ax1.plot(d_range, powercurve_xg(d_range, a_hat, b_hat), lw=2, color="orange", 
         label=f"Curva: a={a_hat:.3f}, b={b_hat:.3f}")
ax1.set_xlabel("Distancia longitudinal (m)")
ax1.set_ylabel("Probabilidad de gol")
ax1.set_title(f"Modelo Longitudinal (solo X)\nR² = {r2_long:.3f}")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Panel derecho: Modelo Euclidiano
ax2 = axes[1]
ax2.scatter(fit_bins_euclid["d_mid"], fit_bins_euclid["p_goal"], s=40, alpha=0.7, label="Datos observados")
ax2.plot(d_range, powercurve_xg(d_range, a_hat_euclid, b_hat_euclid), lw=2, color="green",
         label=f"Curva: a={a_hat_euclid:.3f}, b={b_hat_euclid:.3f}")
ax2.set_xlabel("Distancia euclidiana (m)")
ax2.set_ylabel("Probabilidad de gol")
ax2.set_title(f"Modelo Euclidiano (X + Y)\nR² = {r2_euclid:.3f}")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Mejora: Decaimiento Exponencial

La curva de potencia tiene un problema: puede predecir probabilidades > 1.0 en distancias muy cortas.

El **decaimiento exponencial** soluciona esto, garantizando que la probabilidad siempre sea < 1.0:

```
xG = e^(-d/k) × a + b
```

Donde:
- `d` = distancia euclidiana
- `k` = constante de decaimiento (controla la forma de la curva)
- `a` = multiplicador
- `b` = intercepto

In [ ]:
def exp_decay_xg(d, k, a, b):
    """Modelo de decaimiento exponencial: xG = e^(-d/k) * a + b"""
    return np.exp(-d / k) * a + b

# Ajustar parámetros usando los bins euclidianos
d_data = fit_bins_euclid["d_mid"].values
p_data = fit_bins_euclid["p_goal"].values
w_data = fit_bins_euclid["shots"].values

# Valores iniciales razonables
p0 = [5.0, 0.9, 0.03]  # k, a, b

# Optimizar con pesos (más tiros = más peso)
popt, pcov = curve_fit(exp_decay_xg, d_data, p_data, p0=p0, sigma=1/np.sqrt(w_data), maxfev=5000)
k_opt, a_opt, b_opt = popt

print(f"Parámetros optimizados:")
print(f"  k = {k_opt:.4f} (constante de decaimiento)")
print(f"  a = {a_opt:.4f} (multiplicador)")
print(f"  b = {b_opt:.4f} (intercepto)")
print(f"\nEcuación: xG = e^(-d/{k_opt:.2f}) × {a_opt:.4f} + {b_opt:.4f}")

In [ ]:
# Comparación de R² entre los tres modelos
p_pred_exp = exp_decay_xg(d_data, k_opt, a_opt, b_opt)
r2_exp = r2_score(p_data, p_pred_exp)

# Verificar comportamiento en d=0
xg_at_zero = exp_decay_xg(0, k_opt, a_opt, b_opt)

print("=" * 60)
print("COMPARACIÓN DE MODELOS (distancia euclidiana)")
print("=" * 60)
print(f"\n1. Curva de Potencia:")
print(f"   xG = {a_hat_euclid:.4f} × dist^{b_hat_euclid:.4f}")
print(f"   R² = {r2_euclid:.4f}")
print(f"\n2. Decaimiento Exponencial:")
print(f"   xG = e^(-d/{k_opt:.2f}) × {a_opt:.4f} + {b_opt:.4f}")
print(f"   R² = {r2_exp:.4f}")
print(f"\nMejora en R²: +{(r2_exp - r2_euclid):.4f} ({((r2_exp - r2_euclid) / r2_euclid * 100):.2f}%)")
print(f"\n✓ Verificación: xG en d=0: {xg_at_zero:.4f} (debe ser < 1.0)")

In [ ]:
# Visualización comparativa: Curva de Potencia vs Decaimiento Exponencial
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Extender el rango para mostrar comportamiento cerca de d=0
d_range_full = np.linspace(0.5, D_MAX, 300)

# Panel izquierdo: Curva de Potencia
ax1 = axes[0]
ax1.scatter(d_data, p_data, s=40, alpha=0.7, label="Datos observados")
ax1.plot(d_range_full, powercurve_xg(d_range_full, a_hat_euclid, b_hat_euclid), lw=2, color="green",
         label=f"Potencia: R²={r2_euclid:.3f}")
ax1.axhline(1.0, ls="--", color="red", alpha=0.5, label="Límite p=1.0")
ax1.set_xlabel("Distancia euclidiana (m)")
ax1.set_ylabel("Probabilidad de gol")
ax1.set_title("Curva de Potencia\n(puede exceder 1.0 en distancias cortas)")
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.1)

# Panel derecho: Decaimiento Exponencial
ax2 = axes[1]
ax2.scatter(d_data, p_data, s=40, alpha=0.7, label="Datos observados")
ax2.plot(d_range_full, exp_decay_xg(d_range_full, k_opt, a_opt, b_opt), lw=2, color="purple",
         label=f"Exponencial: R²={r2_exp:.3f}")
ax2.axhline(1.0, ls="--", color="red", alpha=0.5, label="Límite p=1.0")
ax2.scatter([0], [xg_at_zero], s=100, color="orange", marker="*", zorder=5, label=f"d=0: {xg_at_zero:.2f}")
ax2.set_xlabel("Distancia euclidiana (m)")
ax2.set_ylabel("Probabilidad de gol")
ax2.set_title("Decaimiento Exponencial\n(siempre < 1.0)")
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

In [ ]:
# Actualizar df_long con xG usando decaimiento exponencial
df_long["dist_euclid_m"] = df_euclid["dist_euclid_m"]
df_long["xg_euclid"] = np.clip(powercurve_xg(df_long["dist_euclid_m"], a_hat_euclid, b_hat_euclid), 1e-9, 1 - 1e-9)
df_long["xg_exp"] = np.clip(exp_decay_xg(df_long["dist_euclid_m"], k_opt, a_opt, b_opt), 1e-9, 1 - 1e-9)

# Usar modelo exponencial como xG final
df_long["xg_wls"] = df_long["xg_exp"]

# Comparación rápida de xG por tiro
print("Comparación de xG por tiro (muestra):")
print(df_long[["player", "dist_euclid_m", "xg_euclid", "xg_exp", "is_goal"]].sample(5).round(3))

# Recalcular TODAS las agregaciones con el modelo exponencial
# 1) Por partido-equipo
team_match_xg_open = (
    df_long.groupby(["match_id", "team"], as_index=False)
           .agg(
               shots_open_play=("is_goal", "size"),
               goals_open_play=("is_goal", "sum"),
               xg_open_play=("xg_wls", "sum"),
           )
)

# 2) Por equipo-temporada
team_season_xg_open = (
    team_match_xg_open.groupby("team", as_index=False)
                      .agg(
                          matches=("match_id","nunique"),
                          shots_open_play=("shots_open_play","sum"),
                          goals_open_play=("goals_open_play","sum"),
                          xg_open_play=("xg_open_play","sum"),
                      )
)
team_season_xg_open["residual_open_play"] = (
    team_season_xg_open["goals_open_play"] - team_season_xg_open["xg_open_play"]
)

# 3) Crear venue y recalcular división local/visitante
df_long["venue"] = df_long["h_a"].map({"h": "home", "a": "away"})
df_long["venue"] = pd.Categorical(df_long["venue"], categories=["home", "away"], ordered=False)
df_ha = df_long.dropna(subset=["team", "venue"])

home_away_xg_open = (
    df_ha.groupby(["team", "venue"], observed=True)
         .agg(
             matches=("match_id", "nunique"),
             goals_open_play=("is_goal", "sum"),
             xg_open_play=("xg_wls", "sum"),
         )
         .reset_index()
)
home_away_xg_open["xg_per_match_open"] = (
    home_away_xg_open["xg_open_play"] / home_away_xg_open["matches"]
)

home_away_pivot_open = (
    home_away_xg_open
        .pivot(index="team", columns="venue", values="xg_per_match_open")
        .rename(columns={"home": "home_xg_per_match_open", "away": "away_xg_per_match_open"})
        .rename_axis(None, axis=1)
        .reset_index()
        .reindex(columns=["team", "home_xg_per_match_open", "away_xg_per_match_open"])
)

print(f"\n✓ xg_wls actualizado con valores de decaimiento exponencial.")
print(f"✓ team_match_xg_open recalculado: {len(team_match_xg_open)} filas.")
print(f"✓ team_season_xg_open recalculado: {len(team_season_xg_open)} equipos.")
print(f"✓ home_away_pivot_open recalculado: {len(home_away_pivot_open)} equipos.")
print(f"✓ df_ha actualizado: {len(df_ha)} filas.")

In [ ]:
# --- 0) Funciones auxiliares ---
def safe_int(x, default=0):
    """Convertir a int de forma segura; devuelve default en caso de None/NaN/inválido."""
    try:
        if x is None:
            return default
        return int(float(x))  # maneja strings como "2" o "2.0"
    except Exception:
        return default

def _team_name_from_match_side(m: dict, side: str) -> str:
    """Extraer un nombre de equipo estable desde los metadatos del partido de Understat."""
    obj = m.get(side)
    if isinstance(obj, dict):
        for key in ("title", "team_title", "short_title", "name"):
            if key in obj and obj[key]:
                return str(obj[key])
        if "id" in obj and obj["id"] is not None:
            return str(obj["id"])
        return str(obj)
    return "" if obj is None else str(obj)

# --- 1) Marcador oficial por partido-equipo ---
def explode_match_meta(m: dict) -> pd.DataFrame:
    goals = m.get("goals") or {}
    mid = str(m.get("id"))
    h_name = _team_name_from_match_side(m, "h")
    a_name = _team_name_from_match_side(m, "a")
    return pd.DataFrame([
        {"match_id": mid, "team": h_name, "goals_official": safe_int(goals.get("h"), 0)},
        {"match_id": mid, "team": a_name, "goals_official": safe_int(goals.get("a"), 0)},
    ])

official_scores = pd.concat([explode_match_meta(m) for m in raw_matches], ignore_index=True)
official_scores["match_id"] = official_scores["match_id"].astype(str)
official_scores["team"] = official_scores["team"].astype(str)

# --- 2) Usar la agregación OPEN-PLAY de la celda anterior (asegurando llaves como str) ---
team_match_xg_open = (
    team_match_xg_open
    .assign(
        match_id=lambda d: d["match_id"].astype(str),
        team=lambda d: d["team"].astype(str),
    )
)

# --- 3) Merge (solo lo que nos interesa) ---
team_match_minimal = (
    team_match_xg_open[["match_id","team","shots_open_play","goals_open_play","xg_open_play"]]
    .merge(official_scores, on=["match_id","team"], how="left")
)

# ver cuántos goles no fueron open-play
team_match_minimal["non_open_goals"] = (
    (team_match_minimal["goals_official"] - team_match_minimal["goals_open_play"]).clip(lower=0)
)

# --- 4) Verificación: xG vs Goles oficiales para un partido ---
mid = "28960"

# Pivot para ver xG y goles por equipo
match_xg_pivot = (
    team_match_minimal
        .loc[team_match_minimal["match_id"] == mid]
        .pivot(index="match_id", columns="team", values=["xg_open_play", "goals_official"])
        .reset_index()
)

# Aplanar nombres de columnas (MultiIndex → columnas simples)
match_xg_pivot.columns = [
    col if isinstance(col, str) else f"{col[0]}_{col[1]}"
    for col in match_xg_pivot.columns
]

print(match_xg_pivot)

In [ ]:
# Construye una matriz por partido con xG de JUEGO ABIERTO (OPEN-PLAY) 
# para local y visitante.
# Requiere: df_ha (team, venue, match_id, xg_wls) ya creado arriba.

def build_match_xg_matrix(df_ha):
    """Agregar xG por partido, equipo y condición (local/visitante)."""
    agg = (
        df_ha.groupby(["match_id", "team", "venue"], observed=True, as_index=False)
             .agg(xg_open_play=("xg_wls", "sum"))
    )

    # Subconjunto: equipo local
    home = agg[agg["venue"] == "home"][["match_id", "team", "xg_open_play"]].rename(
        columns={"team": "home_team", "xg_open_play": "home_xg"}
    )
    # Subconjunto: equipo visitante
    away = agg[agg["venue"] == "away"][["match_id", "team", "xg_open_play"]].rename(
        columns={"team": "away_team", "xg_open_play": "away_xg"}
    )

    # Una fila por partido
    mx = home.merge(away, on="match_id", how="inner")

    # Higiene básica: asegurar tipo string
    mx["home_team"] = mx["home_team"].astype(str)
    mx["away_team"] = mx["away_team"].astype(str)
    return mx

match_xg = build_match_xg_matrix(df_ha)
print("Partidos en la matriz:", len(match_xg))
match_xg.sample(3)


In [ ]:
def league_goal_avgs_from_xg(match_xg):
    """Calcular el promedio de xG por partido a nivel de liga para locales y visitantes."""
    home_avg = float(match_xg["home_xg"].mean())
    away_avg = float(match_xg["away_xg"].mean())
    return {"promedio_liga_local_xg": home_avg, "promedio_liga_visitante_xg": away_avg}

league_avgs = league_goal_avgs_from_xg(match_xg)
league_avgs

In [ ]:
def team_goal_avgs_from_xg(match_xg, *, home_team: str, away_team: str):
    """
    Calcular el xG promedio por equipo a favor y en contra, 
    separado por condición de local o visitante, para un partido específico.
    Todos los valores corresponden a xG de JUEGO ABIERTO (open-play).
    Retorna:
        {
          "home_for": float, "home_against": float,
          "away_for": float, "away_against": float
        }
    """
    # Equipo local: partidos donde este equipo jugó como local
    home_matches = match_xg[match_xg["home_team"] == home_team]
    # A favor = su propio home_xg; En contra = away_xg del rival
    home_for = float(home_matches["home_xg"].mean()) if len(home_matches) else np.nan
    home_against = float(home_matches["away_xg"].mean()) if len(home_matches) else np.nan

    # Equipo visitante: partidos donde este equipo jugó como visitante
    away_matches = match_xg[match_xg["away_team"] == away_team]
    # A favor = su propio away_xg; En contra = home_xg del rival
    away_for = float(away_matches["away_xg"].mean()) if len(away_matches) else np.nan
    away_against = float(away_matches["home_xg"].mean()) if len(away_matches) else np.nan

    return {
        "home_for": home_for,
        "home_against": home_against,
        "away_for": away_for,
        "away_against": away_against,
    }

# Helper: mostrar nombres de equipos disponibles
available_teams = pd.unique(pd.concat([match_xg["home_team"], match_xg["away_team"]], ignore_index=True))
pd.Series(available_teams).sort_values()

In [ ]:
def poisson_inputs_from_open_play_xg(match_xg, *, home_team: str, away_team: str):
    """
    Construir las 6 entradas requeridas para una plantilla Poisson, usando xG de JUEGO ABIERTO:
      1) Promedio de goles (xG) de los locales por partido en la liga
      2) Promedio de goles (xG) de los visitantes por partido en la liga
      3) Promedio de goles a favor del equipo local (jugando en casa)
      4) Promedio de goles en contra del equipo local (jugando en casa)
      5) Promedio de goles a favor del equipo visitante (jugando fuera)
      6) Promedio de goles en contra del equipo visitante (jugando fuera)
    Retorna un diccionario con esos valores.
    """
    liga = league_goal_avgs_from_xg(match_xg)
    equipos = team_goal_avgs_from_xg(match_xg, home_team=home_team, away_team=away_team)

    out = {
        "promedio_liga_local": liga["promedio_liga_local_xg"],
        "promedio_liga_visitante": liga["promedio_liga_visitante_xg"],
        "local_a_favor": equipos["home_for"],
        "local_en_contra": equipos["home_against"],
        "visitante_a_favor": equipos["away_for"],
        "visitante_en_contra": equipos["away_against"],
    }
    return {
        k: (None if (v is None or (isinstance(v, float) and np.isnan(v))) else float(v))
        for k, v in out.items()
    }

# Ejemplo (ajusta los nombres según tu dataset)
HOME_TEAM = available_teams[10]
AWAY_TEAM = available_teams[7]

inputs = poisson_inputs_from_open_play_xg(match_xg, home_team=HOME_TEAM, away_team=AWAY_TEAM)
inputs

In [ ]:
def to_pretty_df(inputs: dict, home_team: str, away_team: str, decimals: int = 3):
    """Retorna un DataFrame pequeño redondeado para presentación, con un título del partido."""
    items = [
        ("promedio_liga_local", inputs["promedio_liga_local"]),
        ("promedio_liga_visitante", inputs["promedio_liga_visitante"]),
        ("local_a_favor", inputs["local_a_favor"]),
        ("local_en_contra", inputs["local_en_contra"]),
        ("visitante_a_favor", inputs["visitante_a_favor"]),
        ("visitante_en_contra", inputs["visitante_en_contra"]),
    ]
    df = pd.DataFrame(items, columns=["métrica", "valor"])
    df["valor"] = df["valor"].round(decimals)

    # Imprimir un título simple
    print(f"\n⚽ {home_team} vs {away_team}\n")
    return df

# Ejemplo de uso
to_pretty_df(inputs, home_team=HOME_TEAM, away_team=AWAY_TEAM, decimals=3)